# All Sell-out

In [1]:
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.engine import URL
from datetime import datetime
import pandas as pd

In [2]:

class sql_conn:
    def __init__(self) -> None:
        pass

    def setup_connection():
        server = 'mf-enterprise-dev-sql.46ac3df1733c.database.windows.net'
        database = 'PWRAPPDB'
        driver = '{ODBC Driver 18 for SQL Server}'

        # Establish the database connection using AAD Integrated Authentication
        conn_str = (
            f'DRIVER={driver};'
            f'SERVER=tcp:{server};'
            f'DATABASE={database};'
            'Authentication=ActiveDirectoryIntegrated'
        )

        cnxn = pyodbc.connect(conn_str)
        cursor = cnxn.cursor()

        return cnxn, cursor

In [3]:
class us_sellout_compile:
    def __init__(self) -> None:
        self.data = self.pull_sql()
        #self.data_import = self.filter_rows()
        #self.delete_rows()
        #self.insert_rows()


    def pull_sql(self):
        sql_string = """
        WITH sellout_data as (
            SELECT
            'Not Available' as [City], 
            [State], 
            [Segmentation L1], 
            [Segmentation L2], 
            [Restaurants],
            [Consolidated Category], 
            [Distributor], 
            [Week Beginning Date],
            SUM(LBS) as LBS
            FROM [PWRAPPDB].[na_dist].[NA_Distributor_Sellout] as a1
            GROUP BY
            [City], 
            [State], 
            [Segmentation L1], 
            [Segmentation L2], 
            [Restaurants],
            [Consolidated Category], 
            [Distributor], 
            [Week Beginning Date]
        )

        SELECT * FROM sellout_data

        """

        cnxn, cursor = sql_conn.setup_connection()

        df = pd.read_sql(sql_string, cnxn)

        # Commit the transactions
        cnxn.commit()

        # Close the connection
        cursor.close()
        cnxn.close()

        return df
    

    def setup_miengine():
        server = 'mf-enterprise-dev-sql.46ac3df1733c.database.windows.net'
        database = 'PWRAPPDB'
        driver = '{ODBC Driver 18 for SQL Server}'

        # Establish the database connection using AAD Integrated Authentication
        conn_str = URL.create(
            'mssql+pyodbc',
            query={
                'odbc_connect':(
                    f'DRIVER={driver};'
                    f'SERVER=tcp:{server};'
                    f'DATABASE={database};'
                    'Authentication=ActiveDirectoryIntegrated;'
            )
            }
        )

        engine = create_engine(conn_str, connect_args={"autocommit": True}, fast_executemany=True, use_insertmanyvalues=False)

        return engine

    

In [11]:
us_sellout = us_sellout_compile().data

C:\Users\NEWATTER\AppData\Local\Temp\1\ipykernel_53336\4171609775.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_string, cnxn)


In [12]:
from datetime import datetime, timedelta

us_sellout['Week Beginning Date'] = pd.to_datetime(us_sellout['Week Beginning Date'])
us_sellout['week ending (sat)'] = us_sellout['Week Beginning Date'].apply(lambda x: x + pd.DateOffset(days=6))

us_sellout.drop(columns=['Week Beginning Date'], inplace=True)

rename_columns = {
    'City':'city', 
    'State':'region',
    'Segmentation L1':'covid segmentation - l1',
    'Segmentation L2':'covid segmentation - l2', 
    'Restaurants':'restaurant service type',
    'Consolidated Category':'consolidated category',
    'Distributor':'distributor',  
    'LBS':'lbs'
}

us_sellout.rename(columns=rename_columns, inplace=True)

us_sellout.head(3)

,city,region,covid segmentation - l1,covid segmentation - l2,restaurant service type,consolidated category,distributor,lbs,week ending (sat)
0,Not Available,TX,Bars & Pubs,Independents (IOs) / Local Eateries / Takeaway,Restaurants,Prepared Foods,Ben E Keith,15.0,2019-05-25
1,Not Available,PA,Casual Dining,Independents (IOs) / Local Eateries / Takeaway,Restaurants,Potato,PFG,1149.0,2022-09-03
2,Not Available,AZ,QSR / Fast Casual,National Accounts,Restaurants,Potato,PFG,77.0,2024-09-21


In [13]:
ca_sellout = pd.read_csv(r'C:\Users\newatter\OneDrive - McCain Foods Limited\Data Update\files\sellout_region_sysco_ca.zip')

ca_sellout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16165 entries, 0 to 16164
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   City                     0 non-null      float64
 1   Region                   0 non-null      float64
 2   COVID Segmentation - L1  16165 non-null  object 
 3   COVID Segmentation - L2  16165 non-null  object 
 4   Restaurant Service Type  16165 non-null  object 
 5   Consolidated Category    16165 non-null  object 
 6   Calendar Week Year       16165 non-null  float64
 7   Distributor              16165 non-null  object 
 8   LBS                      16165 non-null  float64
 9   SMA_4                    16165 non-null  float64
 10  SMA_8                    16165 non-null  float64
 11  SMA_12                   16165 non-null  float64
 12  YOY Week                 16165 non-null  int64  
 13  LBS_LY                   16165 non-null  float64
 14  SMA_4_LY              

In [14]:
columns_to_keep = [
    'City', 'Region', 'COVID Segmentation - L1', 'COVID Segmentation - L2',
    'Restaurant Service Type', 'Consolidated Category',
    'Distributor', 'LBS', 'Week Ending (Sat)'
]

rename_columns = {
    'City':'city', 
    'Region':'region',
    'COVID Segmentation - L1':'covid segmentation - l1',
    'COVID Segmentation - L2':'covid segmentation - l2', 
    'Restaurant Service Type':'restaurant service type',
    'Restaurants Sub ':'restuarants sub', 
    'Consolidated Category':'consolidated category',
    'Distributor':'distributor', 
    'LBS':'lbs',
    'Week Ending (Sat)':'week ending (sat)'
}

In [15]:
ca_sellout_clean = ca_sellout[columns_to_keep].rename(columns=rename_columns)

ca_sellout_clean.head(3)

,city,region,covid segmentation - l1,covid segmentation - l2,restaurant service type,consolidated category,distributor,lbs,week ending (sat)
0,NaN,NaN,Accommodation,"Hotels, Casinos & Resorts",Not Applicable,Potato,Sysco CA,83768.28,2019-01-05
1,NaN,NaN,Accommodation,"Hotels, Casinos & Resorts",Not Applicable,Potato,Sysco CA,75071.63,2019-01-12
2,NaN,NaN,Accommodation,"Hotels, Casinos & Resorts",Not Applicable,Potato,Sysco CA,87217.60,2019-01-19


In [16]:
sellout = pd.concat([us_sellout, ca_sellout_clean])

sellout['week ending (sat)'] = pd.to_datetime(sellout['week ending (sat)'])

sellout.head(3)

,city,region,covid segmentation - l1,covid segmentation - l2,restaurant service type,consolidated category,distributor,lbs,week ending (sat)
0,Not Available,TX,Bars & Pubs,Independents (IOs) / Local Eateries / Takeaway,Restaurants,Prepared Foods,Ben E Keith,15.0,2019-05-25
1,Not Available,PA,Casual Dining,Independents (IOs) / Local Eateries / Takeaway,Restaurants,Potato,PFG,1149.0,2022-09-03
2,Not Available,AZ,QSR / Fast Casual,National Accounts,Restaurants,Potato,PFG,77.0,2024-09-21


In [17]:
sellout.to_csv(r'C:\Users\newatter\OneDrive - McCain Foods Limited\North America\SELLOUT_REGION.csv', index=False)